## gcForest


项目官网：http://lamda.nju.edu.cn/code_gcForest.ashx?AspxAutoDetectCookieSupport=1   
项目地址：https://github.com/kingfengji/gcForest   
学习资料：http://blog.csdn.net/sinat_29819401/article/details/60160755   
原始论文：https://arxiv.org/abs/1702.08835v2

* 输入数据是python矩阵。特征X和标签y需在代码里加载调整，支持arff等固定格式也可以，但不方便调参。   
* 训练输出是用于预测的model文件，训练过程中只有Accuracy指标。   
* gcForest现在不支持GPU加速，只有CPU多线程。虽然模型本身参数不多，但实际跑的时间还是很长（minist手写图片识别样例在131上训练了一个多小时才结束）。如果后面程序优化好了或者支持GPU了，可能会改善很多。   
* 效果也并不十分突出，minist手写图片识别样例的最终准确率约在88%左右，与最基本的全连接DNN性能差不多，与CNN（约99%）相差很大。   

总结：我觉得这个方法目前还是开创性工作，把神经元换成了随机森林，也就去除了反向传播，属于深度学习的“替代”方案，但还需要大家加入更多创新点改造，才有比较满意的结果。   

如果要用到自己的数据里，需注意根据自己数据的特征维度更改model.json中网络的矩阵shape。


In [1]:
# coding: utf-8

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets.samples_generator import make_classification

from gcforest.utils.config_utils import load_json
from gcforest.fgnet import FGNet, FGTrainConfig

# 删除缓存,shell: !rm -r cache
import shutil
shutil.rmtree('cache')

# 载入数据集，1000个样本，400维特征，10个标签，8种信息
X, y = make_classification(n_samples=1000, n_features=400, n_classes=10, n_informative=8)
# 变换X的维度为1000*1*20*20，具体与model.json的参数有关
X = X.reshape(-1, 1, 20, 20)
# 分割训练集与测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2017)

# 载入网络配置文件，与caffe类似
config = load_json('model.json')
train_config = FGTrainConfig(config["train"])

# 开始训练
net = FGNet(config["net"], train_config.data_cache)
net.fit_transform(X_train, y_train, X_test, y_test, train_config)

[ 2017-09-29 13:21:07,962][fgnet.fit_transform] X_train.shape=(800, 1, 20, 20), y_train.shape=(800,), X_test.shape=(200, 1, 20, 20), y_test.shape=(200,)
[ 2017-09-29 13:21:07,963][fg_win_layer.fit_transform] [data][win1/7x7], bottoms=[u'X', u'y'], tops=[u'win1/7x7/ets', u'win1/7x7/rf']
[ 2017-09-29 13:21:07,964][fg_win_layer.fit_transform] [progress][win1/7x7] ti=0/2, top_name=win1/7x7/ets
[ 2017-09-29 13:21:07,966][fg_win_layer.fit_transform] [data][win1/7x7,train] bottoms.shape=800x1x20x20,800
[ 2017-09-29 13:21:07,967][win_utils.get_windows] get_windows_start: X.shape=(800, 1, 20, 20), X_win.shape=(49, 39200), nw=7, nh=7, c=1, win_x=7, win_y=7, stride_x=2, stride_y=2
[ 2017-09-29 13:21:08,107][win_utils.get_windows] get_windows_end
[ 2017-09-29 13:21:08,109][fg_win_layer.fit_transform] [data][win1/7x7,test] bottoms.shape=200x1x20x20,200
[ 2017-09-29 13:21:08,110][win_utils.get_windows] get_windows_start: X.shape=(200, 1, 20, 20), X_win.shape=(49, 9800), nw=7, nh=7, c=1, win_x=7, win